# Computational Theory Assessment
---

In [1]:
# Import necessary libraries
import numpy as np

## Problem 1: Binary Words and Operations

### Implement the following functions in Python. Use **NumPy** to ensure that all variables and values are treated as **32-bit integers**. These functions are defined in the **Secure Hash Standard (see page 10)**.

1. `Parity(x, y, z)`
2. `Ch(x, y, z)`
3. `Maj(x, y, z)`
4. `Sigma0(x)` - *written as $\Sigma_0^{256}(x)$ in the standard.*
5. `Sigma1(x)` - *written as $\Sigma_1^{256}(x)$ in the standard.*
6. `sigma0(x)` - *written as $\sigma_0^{256}(x)$ in the standard.*
7. `sigma1(x)` - *written as $\sigma_1^{256}(x)$ in the standard.*

### Instructions

- Document each function with a clear docstring
- Explain its purpose and behaviour in Markdown
- Test it with appropriate examples to verify correctness.

### Solution:

**1. Parity(x, y, z) Function (Used in SHA-1 - deprecated)**

The Parity function is a linear bitwise operation used in certain rounds of the SHA-1 hash algorithm (specifically rounds 20-39 and 60-79). It takes three 32-bit words, which we can call `x`, `y`, and `z`, and combines them by applying the XOR operation to each corresponding bit. 

$$
Parity(x, y, z) = x \oplus y \oplus z
$$

XOR (exclusive OR) works like this: for each bit position, if the number of `1`s in that position across `x`, `y`, and `z` is **odd**, the result bit is `1`. If it is **even**, then the result bit is `0`. 

This function helps increase **diffusion** in the hash algorithm, meaning that small changes in input spread out and affect many bits in the output. This contributes to the security of SHA-1 by making it more resistant to certain types of cryptographic attacks.

However, **SHA-1 is no longer considered secure** due to successful **collision attacks**. Beginning around 2011, major organizations started phasing it out. Today, SHA-1 is officially deprecated in favor of stronger and more secure algorithms like SHA-256 and SHA-3, neither of which use the Parity function.

In [2]:
def parity(x: int, y: int, z: int) -> np.uint32:
    """
    Performs a bitwise XOR (parity function) across three 32-bit integers.

    This function is used in the SHA-1 algorithm during rounds 20-39 and 60-79.

    Parameters:
        x (int): The first 32-bit integer.
        y (int): The second 32-bit integer.
        z (int): The third 32-bit integer.

    Returns:
        np.uint32: The result of x XOR y XOR z, as a 32-bit unsigned integer.
    """

    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return x ^ y ^ z


In [3]:
# Test cases to validate the Parity function
def test_parity():
    test_cases = [
        # Basic tests (decimal)
        ((0, 0, 0), 0), 
        ((1, 0, 0), 1),
        ((1, 1, 0), 0),
        ((1, 1, 1), 1),

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b0000),
        ((0b1100, 0b0011, 0b0101), 0b1010),
        ((0b1111, 0b1111, 0b1111), 0b1111),

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0x00000000, 0x00000000), 0xFFFFFFFF),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0x00000000), 0x00000000),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF), 0xFFFFFFFF),
        ((0xAAAAAAAA, 0x55555555, 0xFFFFFFFF), 0x00000000),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]

    for (x, y, z), expected in test_cases:
        result = parity(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for parity({hex(x)}, {hex(y)}, {hex(z)})"
        )

    # Final complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = np.uint32(x) ^ np.uint32(y) ^ np.uint32(z)
    result = parity(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for parity({hex(x)}, {hex(y)}, {hex(z)})"
    )

    print("All parity function test cases passed.")

test_parity()

All parity function test cases passed.


**2. Ch(x, y, z) Function (Used in SHA-1 and all SHA-2 variants)**

The Ch (Choice) function is a nonlinear bitwise operation used in both SHA-1 (specifically in rounds 0-19) and all SHA-2 variants. It takes three 32-bit (or 64-bit, depending on the SHA-2 variant) words as input: `x`, `y`, and `z`.

$$
Ch(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

The Ch function operates as a bit selector or multiplexer:
- For each bit position, if the bit in `x` is `1`, the function selects the corresponding bit from `y`
- If the bit in `x` is `0`, the function selects the corresponding bit from `z`

This provides important **nonlinear mixing** in the hash algorithm. Unlike the Parity function which is linear, the Ch function has nonlinear behavior that strengthens the cryptographic properties of SHA algorithms by making them more resistant to differential cryptanalysis and other attacks.

Although **SHA-1 is deprecated** due to proven collision vulnerabilities, the `Ch` function remains an essential part of **SHA-2**, which is still considered secure and widely used today.

The name "Ch" reflects its behavior as a "chooser" or "selector" between the bits of `y` and `z` based on `x`.

In [4]:
def ch(x: int, y: int, z: int) -> np.uint32:
    """
    Performs the Choice function on three 32-bit integers.
    
    This function is used in:
    - SHA-1 algorithm during rounds 0-19 
    - All SHA-2 variants
    
    Parameters:
        x (int): Selector 32-bit integer.
        y (int): Bits to be selected when x bit is 1.
        z (int): Bits to be selected when x bit is 0.

    Returns:
        np.uint32: The result of (x & y) ^ (~x & z), as a 32-bit unsigned integer.
    """
    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return (x & y) ^ (~x & z)

In [5]:
# Test cases to validate the ch function
def test_ch():
    test_cases = [
        # Basic tests (decimal)
        ((0, 0, 0), 0), 
        ((1, 0, 0), 0),   
        ((1, 1, 0), 1),   
        ((1, 1, 1), 1),   

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b0101),  
        ((0b1100, 0b0011, 0b0101), 0b0001),  
        ((0b1111, 0b1111, 0b1111), 0b1111),  

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0xAAAAAAAA, 0x55555555), 0xAAAAAAAA),
        ((0x00000000, 0xAAAAAAAA, 0x55555555), 0x55555555),
        ((0xAAAAAAAA, 0xFFFFFFFF, 0x00000000), 0xAAAAAAAA),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]
    
    for (x, y, z), expected in test_cases:
        result = ch(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for ch({hex(x)}, {hex(y)}, {hex(z)})"
        )
    
    # Complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = (np.uint32(x) & np.uint32(y)) ^ (~np.uint32(x) & np.uint32(z))
    result = ch(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for ch({hex(x)}, {hex(y)}, {hex(z)})"
    )
    
    print("All ch function test cases passed.")

# Run the tests
test_ch()

All ch function test cases passed.


**3. Maj(x, y, z) Function (Used in SHA-1 and all SHA-2 variants)**

The Maj (Majority) function is a nonlinear bitwise operation used in both SHA-1 (specifically rounds 40-59) and all SHA-2 variants. It takes three 32-bit (or 64-bit, depending on the SHA-2 variant) words as input: `x`, `y`, and `z`.

$$
Maj(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)
$$

The Maj function implements a bit-wise majority vote: for each bit position, the output bit is `1` if at least two of the three input bits are `1`, otherwise it is `0`.

This function provides important **nonlinear mixing** in the hash algorithm. The majority operation effectively contributes to the avalanche effect, where small changes in input bits cause widespread and complex changes in the output. This property is essential for the cryptographic strength of SHA algorithms.

Although **SHA-1 is deprecated** due to collision vulnerabilities, the `Maj` function remains an essential part of **SHA-2**, which is still considered secure and widely used today.

The name "Maj" reflects its behavior as a "majority" selector that chooses the most common bit value among the three inputs at each position.


In [6]:
def maj(x: int, y: int, z: int) -> np.uint32:
    """
    Performs the Majority function on three 32-bit integers.
    
    This function is used in:
    - SHA-1 algorithm during rounds 40-59
    - All SHA-2 variants
    
    Parameters:
        x (int): The first 32-bit integer.
        y (int): The second 32-bit integer.
        z (int): The third 32-bit integer.

    Returns:
        np.uint32: The result of (x & y) ^ (x & z) ^ (y & z), as a 32-bit unsigned integer.
    """
    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return (x & y) ^ (x & z) ^ (y & z)  

In [7]:
# Test cases to validate the Maj function
def test_maj():
    test_cases = [
        # Basic tests
        ((0, 0, 0), 0),
        ((1, 0, 0), 0),
        ((1, 1, 0), 1),
        ((1, 1, 1), 1),

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b1111),
        ((0b1100, 0b0011, 0b0101), 0b0101),
        ((0b1111, 0b1111, 0b1111), 0b1111),

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0x00000000, 0x00000000), 0x00000000),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0x00000000), 0xFFFFFFFF),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF), 0xFFFFFFFF),
        ((0xAAAAAAAA, 0x55555555, 0xFFFFFFFF), 0xFFFFFFFF),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]
    
    for (x, y, z), expected in test_cases:
        result = maj(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for maj({hex(x)}, {hex(y)}, {hex(z)})"
        )
    
    # Complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = (np.uint32(x) & np.uint32(y)) ^ (np.uint32(x) & np.uint32(z)) ^ (np.uint32(y) & np.uint32(z))
    result = maj(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for maj({hex(x)}, {hex(y)}, {hex(z)})"
    )
    
    print("All maj function test cases passed.")

# Run the tests
test_maj()

All maj function test cases passed.


**4. Sigma0(x) Function (Used in SHA-256 and SHA-224)**

The `Sigma0(x)` function is a nonlinear bitwise operation used in the SHA-256 and SHA-224 hash algorithms. It takes a single 32-bit word `x` as input.

$$
\Sigma_0^{256}(x) = ROTR^2(x) \oplus ROTR^{13}(x) \oplus ROTR^{22}(x)
$$

Where `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions.

The function takes the input 32-bit word `x` and performs three different circular right rotations (by 2, 13, and 22 bits), then combines the results using the XOR operation. 

This operation provides important **nonlinear diffusion** in the hash algorithm. The combination of different rotation amounts ensures that changes in the input bits propagate widely and unpredictably throughout the output, contributing to the cryptographic strength of SHA-256 and SHA-224.


In [8]:
def big_sigma0(x: int) -> np.uint32:
    """
    Performs the Σ₀ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function 
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input 
    word by 2, 13, and 22 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x), as a 32-bit unsigned integer.
    """
    x = np.uint32(x)

    # Perform the circular right rotations 
    rotr2 = np.uint32((x >> 2) | (x << 30)) # 32 - 2 = 30
    rotr13 = np.uint32((x >> 13) | (x << 19)) # 32 - 13 = 19
    rotr22 = np.uint32((x >> 22) | (x << 10)) # 32 - 22 = 10

    return rotr2 ^ rotr13 ^ rotr22


In [9]:
# Test cases to validate the big_sigma0 function
def test_big_sigma0():
    test_cases = [
        # Basic tests
        (0x00000000, 0x00000000), # ROTR of all 0s XORed = all 0s
        (0xFFFFFFFF, 0xFFFFFFFF), # ROTR of all 1s XORed = all 1s
        (0x00000001, big_sigma0(0x00000001)),
        (0x80000000, big_sigma0(0x80000000)),

        # Bit pattern tests (binary)
        (0b00000000000000000000000000001111, big_sigma0(0b00000000000000000000000000001111)),
        (0b11110000000000000000000000000000, big_sigma0(0b11110000000000000000000000000000)),

        # Hex patterns
        (0xAAAAAAAA, big_sigma0(0xAAAAAAAA)),
        (0x55555555, big_sigma0(0x55555555)),
        (0xDEADBEEF, big_sigma0(0xDEADBEEF)),
        (0x12345678, big_sigma0(0x12345678)),

        # Edge case
        (0x7FFFFFFF, big_sigma0(0x7FFFFFFF)),
    ]

    for x, expected in test_cases:
        result = big_sigma0(x)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for big_sigma0({hex(x)})"
        )

    print("All big_sigma0 function test cases passed.")

# Run the test
test_big_sigma0()


All big_sigma0 function test cases passed.


**5. Sigma1(x) Function (Used in SHA-256 and SHA-224)**

The `Sigma1(x)` function is a nonlinear bitwise operation used in the SHA-256 and SHA-224 hash algorithms. It takes a single 32-bit word `x` as input.

$$
\Sigma_1^{256}(x) = ROTR^6(x) \oplus ROTR^{11}(x) \oplus ROTR^{25}(x)
$$

Where `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions.

The function takes the input 32-bit word `x` and performs three different circular right rotations (by 6, 11, and 25 bits), then combines the results using the XOR operation.

This operation provides important **nonlinear diffusion** in the hash algorithm. The combination of different rotation amounts ensures that changes in the input bits propagate widely and unpredictably throughout the output, contributing to the cryptographic strength of SHA-256 and SHA-224.


In [10]:
def big_sigma1(x: int) -> np.uint32:
    """
    Performs the Σ₁ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input
    word by 6, 11, and 25 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x), as a 32-bit unsigned integer.
    """
    x = np.uint32(x)

    # Perform the circular right rotations
    rotr6 = np.uint32((x >> 6) | (x << 26))  # 32 - 6 = 26
    rotr11 = np.uint32((x >> 11) | (x << 21))  # 32 - 11 = 21
    rotr25 = np.uint32((x >> 25) | (x << 7))  # 32 - 25 = 7

    return rotr6 ^ rotr11 ^ rotr25

In [11]:
# Test cases to validate the big_sigma1 function
def test_big_sigma1():
    test_cases = [
        # Basic tests
        (0x00000000, 0x00000000), # ROTR of all 0s XORed = all 0s
        (0xFFFFFFFF, 0xFFFFFFFF), # ROTR of all 1s XORed = all 1s
        (0x00000001, big_sigma1(0x00000001)),
        (0x80000000, big_sigma1(0x80000000)),

        # Bit pattern tests (binary)
        (0b00000000000000000000000000001111, big_sigma1(0b00000000000000000000000000001111)),
        (0b11110000000000000000000000000000, big_sigma1(0b11110000000000000000000000000000)),

        # Hex patterns
        (0xAAAAAAAA, big_sigma1(0xAAAAAAAA)),
        (0x55555555, big_sigma1(0x55555555)),
        (0xDEADBEEF, big_sigma1(0xDEADBEEF)),
        (0x12345678, big_sigma1(0x12345678)),

        # Edge case
        (0x7FFFFFFF, big_sigma1(0x7FFFFFFF)),
    ]

    for x, expected in test_cases:
        result = big_sigma1(x)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for big_sigma1({hex(x)})"
        )

    print("All big_sigma1 function test cases passed.")

# Run the test
test_big_sigma1()

All big_sigma1 function test cases passed.


**6. sigma0(x) Function (Used in SHA-256 and SHA-224)**

The `sigma0(x)` function is a nonlinear bitwise operation used in the SHA-256 and SHA-224 hash algorithms. It operates on a single 32-bit word and is used in the message schedule.

$$
\sigma_0^{256}(x) = ROTR^7(x) \oplus ROTR^{18}(x) \oplus SHR^3(x)
$$

Where `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions, and `SHR^n(x)` represents a right shift of `x` by `n` positions (filling with zeros on the left).

The function takes the input 32-bit word `x` and performs two circular right rotations (by 7 and 18 bits) and one right shift (by 3 bits), then XORs all three values together.

This function is used in the message schedule expansion (section 6.2.2 of the Secure Hash Standard) in SHA-256 and SHA-224. It provides crucial **nonlinear diffusion** in the algorithm. The combination of rotations and shifts ensures that bit changes propagate throughout the word in complex patterns, contributing to the cryptographic strength of SHA-256.


In [ ]:
def small_sigma0(x: int) -> np.uint32:
    """
    Performs the σ₀ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 7 and 18 positions, one right shift (SHR) by 3 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x), as a 32-bit unsigned integer
    """
    # Ensure input is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Perform circular right rotations by 7 and 18 bits
    rotr7 = np.uint32((x >> 7) | (x << 25)) # 32 - 7 = 25
    rotr18 = np.uint32((x >> 18) | (x << 14)) # 32 - 18 = 14

    # Perform right shift by 3 bits
    # SHR^n(x) = x >> n (fills with zeros on the left)
    shr3 = np.uint32(x >> 3)
    
    # XOR all three values together
    return rotr7 ^ rotr18 ^ shr3


---

## Problem 2: Fractional Parts of Cube Roots

### Use **NumPy** to calculate the constants listed at the bottom of **page 11 of the Secure Hash Standard**, following the steps below. These are the **first 32 bits of the fractional parts** of the **cube roots of the first 64 prime numbers**.

### Instructions

- Write a function called `primes(n)` that generates the first `n` prime numbers.
- Use the function to calculate the **cube root** of the first 64 primes.
- For each cube root, extract the first thirty-two bits of the fractional part.
- Display the result in **hexadecimal**.
- Test the results against what is in the Secure Hash Standard.

### Solution:

---

## Problem 3: Padding

### Instructions

- Write a generator function `block_parse(msg)` that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. 
- The function should accept a bytes object called `msg`. 
- At each iteration, it should yield the next 512-bit block of `msg` as a bytes object. 
- Ensure that the final block (or final two blocks) include the required padding of `msg` as specified in the standard. 
- Test the generator with messages of different lengths to confirm proper padding and block output.

### Solution:

---

## Problem 4: Hashes

### Instructions

- Write a function `hash(current, block)` that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

### Solution:

---

## Problem 5: Passwords

### Instructions

- The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. 
- As strings, they were encoded using UTF-8. 
- Determine the passwords and explain how you found them. 
- Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

### SHA-256 hashes 
1. 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
2. 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
3. b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342

### Solution:

---
# End of Assessment